In [ ]:
import pandas as pd
import plotly.graph_objects as go
import json

df = pd.read_csv("202501_202507_주민등록인구및세대현황_월간.csv", encoding='cp949')
df

In [62]:
MONTH = [f"{i:02}월" for i in range(1, 8)]
TYPE = ['총인구수','남자 인구수', '여자 인구수']
for M in MONTH:
    for T in TYPE:
    # df의 "2025년01월~07월, _총인구수, 남자 인구수, 여자 인구수" 열의 데이터를 정수로 변환
        df[f'2025년{M}_{T}'] = df[f'2025년{M}_{T}'].apply(lambda x: int(x.replace(',','')))
        df = df[df[f'2025년{M}_{T}'] != 0]

# '행정구역'열 => '구역명','구역코드'로 분할
def split_code_name(x):
    splited = x.split('(')
    return pd.Series([splited[0].strip(), splited[1][0:5]])

df[['구역명','구역코드']] = df['행정구역'].apply(lambda x: split_code_name(x))
df['시도명'] = df['구역명'].apply(lambda x: x.split()[0])
# 시도명 축약(서울특별시 -> 서울, 경상남도 -> 경남)
def abbreviate_province(name):
    if name[-2:] in ['남도','북도']:
        return name[0] + name[2]
    else:
        return name[:2]
df['시도명(함축)'] = df['시도명'].apply(lambda x : abbreviate_province(x))

C:\Users\turing07\AppData\Local\Temp\ipykernel_22800\3301256088.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [63]:
geojson = json.load(open('법정구역_시군구.geojson','r',encoding='utf8'))
sido_geojson = {feat['properties']['SIG_CD']: feat['properties']['SIG_KOR_NM'] for feat in geojson['features']}
df['구역명확인'] = df['구역코드'].map(sido_geojson)
# GeoJSON에 없는 구역들은 삭제해줍니다.
df = df[df['구역명확인'].notna()].copy().reset_index(drop=True)


In [64]:
COLOR_SCALES = {
    '총인구수': [[0, '#F1EFEF'], [0.2, '#CCC8AA'],   [0.4, '#B0AA7C'],  [0.6, '#7D7C7C'], [0.8,'#606060'], [1, '#191717']],
    '남자 인구수': [[0,"#EFEFF1"],[0.2,"#9D9DBD"],[0.4,"#5252AE"],[0.6,"#21217D"],[0.8,"#08083B"],[1,"#010115"]],
    '여자 인구수': [[0,'#F1EFEF'],[0.2,"#F4B0B0"],[0.4,"#9A3434"],[0.6,"#891212"],[0.8,"#470B0B"],[1,"#040000"]]
                }
# 단계구분도를 반환하는 함수
def getChoropleth(geojson, featureidkey, locations, z, colorscale, hovertext,visible=True):
    return go.Choropleth(geojson=geojson,
                                    featureidkey=featureidkey,#GeoJSON에서 활용할 id키
                                    locationmode= 'geojson-id',
                                    locations=locations,
                                    z = z,
                                    colorscale = colorscale,
                                    # 컬러스케일은 원하는 갯수의 색상으로 구성할 수 있습니다.
                                    hovertext = hovertext,
                                    hoverinfo = 'text',
                                    marker_line_color='#555',
                                    # marker_line : 행정구역의 경계선
                                    marker_line_width=1,     
                                    visible=visible                       
                                    )





In [65]:
#월별 슬라이더의 프레임을 구성하기 위해 각 월에 대한 트레이스 준비
#슬라이더의 각 스텝과 프레임 이름을 매칭 하기 위한 월 라벨 목록
frames = []
init_visible = dict(zip(TYPE, [True,False,False]))
for M in MONTH:
    frames.append(go.Frame(
        name=M,         #프레임 고유 이름, 슬라이더 step의 첫번째 요소와 동일한 값이어야 함
        data=[         #프레임에서 표현할 트레이스
            getChoropleth(geojson,
                          'properties.SIG_CD',
                          locations=df['구역코드'],
                          z= df[f'2025년{M}_{T}'],
                          colorscale = COLOR_SCALES[T],
                          # 컬러스케일은 원하는 갯수의 색상으로 구성할 수 있습니다.
                          hovertext = df['구역명확인']+": " +df[f'2025년{M}_{T}'].apply(lambda x: f'{x:,} 명'),               
            ) for T in TYPE]
    ))
    
    

#전체 트레이스 컨테이너, 초기 data, frames, layout 할당
fig = go.Figure(
    data=[getChoropleth(geojson,
        'properties.SIG_CD',
        locations=df['구역코드'],
        z= df[f'2025년01월_{T}'],
        colorscale = COLOR_SCALES[T],
        # 컬러스케일은 원하는 갯수의 색상으로 구성할 수 있습니다.
        hovertext = df['구역명확인']+": " +df[f'2025년{M}_{T}'].apply(lambda x: f'{x:,} 명'),
        ) for T in TYPE ],
    frames=frames
)

#슬라이더 (월 이동)
sliders = [{
    'active':0,                 #처음 선택할 step의 인덱스
    'x': 0.08, 'y': -0.12,      #하단 배치
    'len': 0.84,                #슬라이더 길이
    'currentvalue':{            #현재 값 표시 영역, prefix/suffix: 값의 앞/뒤에 이어 붙일 문장 작성
        'prefix': '월: '
    },
    'steps':[{                  #슬라이더의 각 칸을 의미하는 리스트
        'method': 'animate',    #실행할 동작
        'args':[                #각 step의 세부 설정
            [M],                #각 재생할 프레임 목록, 이름이 M인 프레임으로 전환, [None]으로 할당시 순서대로 재생
            {  
                'mode': 'immediate',
                'frame':{'duration':0, 'redraw': True}, # duration: 프레임 간격(밀리초, 1000 = 1초), redraw: 매번 전체 트레이스 다시 그리기,
                                                        #False일 경우 변한 부분만 다시 그리기
            }
        ],
        'label':M
    } for M in MONTH]
}]



In [66]:
play_pause = {
    "type":"buttons",
    "direction":'right',
    'x':0.0, 'y': -0.02, 'xanchor':"left", 'yanchor':'top',
    "buttons":[{
        "label":"▶ 재생",
        "method": "animate",
        "args":[None, {"frame": {"duration": 1000, "redraw": True}, "fromcurrent": True, "transition": {"duration": 100}}]
    },
    {
        "label":"││ 일시정지",
        "method":"animate",
        "args":[[None], {"frame": {"duration": 0, "redraw": False}, "mode":"immediate"}]                   
    }]
}
#전환 버튼 (총/남/여) - traces의 visible 상태를 제어
# - 각 프레임은 3개의 trace(총/남/여)데이터를 모두 가지고 있고,
# - 버튼은 어떤 trace만 보이게 할지 토글

metric_buttons = {
    'type':"buttons",
    'direction':"right",
    'x':1.0, 'y':1.2, 'xanchor':'right', 'yanchor':'top',
    'showactive': True,
    "buttons":[
        {"label":"총인구", "method":"update", "args":[{"visible":[True, False, False],}]},
        {"label":"남자", "method":"update", "args":[{"visible":[False, True, False],}]},
        {"label":"여자", "method":"update", "args":[{"visible":[False, False, True],}]},
    ]
}

fig.update_layout(
    title='25년 1월 ~ 7월 시군구 별 인구 현황',
    sliders=sliders,
    updatemenus = [ play_pause, metric_buttons,],
    margin={'t':110, 'b':100, 'l':40, 'r':40},
    geo = dict(fitbounds="locations", # 설정하지 않으면 세계지도가 표시됩니다
                             visible = False, # 설정하지 않으면 세계지도가 표시됩니다
                             showframe=False,
                             )
)
html = fig.to_html()
with open('2025년 1월 ~ 7월 시군구별 인구 현황.html', "w", encoding="utf-8") as f:
    f.write(html)
# html 파일로 export